# Linear in means model test

This notebook demonstrates usage of the redesigned adversarial estimator for the linear-in-means model.

In [ ]:
import numpy as np
from structural_gnn_lib import GraphDataset, AdversarialEstimator, linear_in_means_model
from structural_gnn_lib.discriminator.discriminator import GraphDiscriminator

In [ ]:
# 1. Wrangle data
N = 50
p = 0.1
rng = np.random.default_rng(0)
A = (rng.random((N, N)) < p).astype(float)
A = np.triu(A, 1)
A += A.T
X = rng.standard_normal((N, 1))
Y = np.zeros((N, 1))
data = GraphDataset(X=X, Y=Y, A=A, N=list(range(N)))

In [ ]:
# 2. Define discriminator factory
def make_discriminator(input_dim):
    return GraphDiscriminator(input_dim)

In [ ]:
# 3. Structural model mapping is `linear_in_means_model`


In [ ]:
# 4. Hyperparameters for GP minimizer
gp_params = dict(n_calls=10, n_initial_points=5, random_state=0)

In [ ]:
# 5. Use adversarial estimator
estimator = AdversarialEstimator(
    ground_truth_data=data,
    structural_model=linear_in_means_model,
    initial_params=[0.1, 0.1],
    bounds=[(0.0, 1.0), (0.0, 1.0)],
    discriminator_factory=make_discriminator,
    gp_params=gp_params,
)
# result = estimator.estimate(m=20, num_epochs=2)